In [45]:
from warnings import filterwarnings
filterwarnings('ignore')

In [46]:
import pandas as pd
df = pd.read_csv('con_dataset.csv')

In [47]:
df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,Yes,162.0,2.5,1040.0,676.0,28.0,79.99
1,540.0,0.0,Yes,162.0,2.5,1055.0,676.0,28.0,61.89
2,332.5,142.5,Yes,228.0,0.0,NaN,594.0,270.0,40.27
3,332.5,142.5,No,228.0,0.0,932.0,594.0,365.0,41.05
4,198.6,132.4,No,192.0,0.0,978.4,825.5,360.0,44.30


In [48]:
df.isna().sum()

Cement                4
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      4
Fine Aggregate        0
Age                   4
Strength              0
dtype: int64

# Seperat the X and Y

In [49]:
X= df.drop(columns=['Strength'])

In [50]:
Y = df[['Strength']]

In [51]:
Y

,Strength
0,79.99
1,61.89
2,40.27
3,41.05
4,44.30
...,...
1025,44.28
1026,31.18
1027,23.70
1028,32.77


In [52]:
df.dtypes

Cement                float64
Blast Furnace Slag    float64
Fly Ash                object
Water                 float64
Superplasticizer      float64
Coarse Aggregate      float64
Fine Aggregate        float64
Age                   float64
Strength              float64
dtype: object

In [53]:
df.columns

Index(['Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer',
       'Coarse Aggregate', 'Fine Aggregate', 'Age', 'Strength'],
      dtype='object')

In [54]:
cat=list(X.columns[X.dtypes == 'object'])

In [55]:
con = list(X.columns[X.dtypes != 'object'])

In [56]:
cat 

['Fly Ash']

In [57]:
con

['Cement',
 'Blast Furnace Slag',
 'Water',
 'Superplasticizer',
 'Coarse Aggregate',
 'Fine Aggregate',
 'Age']

In [58]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer # Fill the Missing Value
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder 
# Scalar == Scaling Two Type 1) Standard -- X - Xm / SD  --- Range -3 tp 3   2) Normalition -- X - Xmin / Xmax - Xmin  Range - -1 to 1
# OneHot Encoder = Convert Categorical Features into Continous --- Create Multiple Columns -- Each Wrod One Columns e.g Apple --- 
# new columns will get created for Apple ---- where ever there is apple entry it will show 1 otherwise 0 
# Ordinal Encoder - used for Ordinal data like S M L XL , Only One Columns will be get created it will have value like S-1, M-2, L-3, XL-4
from sklearn.compose import ColumnTransformer # Joining the Cat and Con Columns 

In [59]:
#Create Numerical Pipeline 
num_pipe = Pipeline(steps= [("imputer", SimpleImputer(strategy='mean')),('Scalar', StandardScaler()) ] ) 

# Create Categorical Pipeline 
cat_Pipe = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                           ('ohe', OneHotEncoder(handle_unknown='ignore'))])

#Combine categorical and continuous pipelines
pre = ColumnTransformer([('num', num_pipe, con), ('cat', cat_Pipe, cat)])

In [63]:
# Transform the Array 
X_pre = pre.fit_transform(X)
X_pre

array([[ 2.49569686, -0.85688789, -0.91676439, ..., -0.27647427,
         0.        ,  1.        ],
       [ 2.49569686, -0.85688789, -0.91676439, ..., -0.27647427,
         0.        ,  1.        ],
       [ 0.49970827,  0.79552649,  2.17546125, ...,  3.6148215 ,
         0.        ,  1.        ],
       ...,
       [-1.27022859,  0.75957923,  0.52158905, ..., -0.27647427,
         1.        ,  0.        ],
       [-1.16826484,  1.30806485, -0.2795785 , ..., -0.27647427,
         0.        ,  1.        ],
       [-0.18902803,  0.30849909,  0.89171909, ..., -0.27647427,
         0.        ,  1.        ]])

In [66]:
# Get the features / Columns names 
cols = pre.get_feature_names_out()
cols

array(['num__Cement', 'num__Blast Furnace Slag', 'num__Water',
       'num__Superplasticizer', 'num__Coarse Aggregate',
       'num__Fine Aggregate', 'num__Age', 'cat__Fly Ash_No',
       'cat__Fly Ash_Yes'], dtype=object)

In [68]:
# Convert into Data Frame 
X_pre = pd.DataFrame(X_pre, columns=cols)
X_pre

,num__Cement,num__Blast Furnace Slag,num__Water,num__Superplasticizer,num__Coarse Aggregate,num__Fine Aggregate,num__Age,cat__Fly Ash_No,cat__Fly Ash_Yes
0,2.495697,-0.856888,-0.916764,-0.620448,0.862035,-1.217670,-0.276474,0.0,1.0
1,2.495697,-0.856888,-0.916764,-0.620448,1.055123,-1.217670,-0.276474,0.0,1.0
2,0.499708,0.795526,2.175461,-1.039143,0.000000,-2.240917,3.614821,0.0,1.0
3,0.499708,0.795526,2.175461,-1.039143,-0.528203,-2.240917,5.142396,1.0,0.0
4,-0.788306,0.678408,0.488793,-1.039143,0.069084,0.647884,5.061998,1.0,0.0
...,...,...,...,...,...,...,...,...,...
1025,-0.039930,0.488235,-0.092171,0.451410,-1.325016,-0.065893,-0.276474,0.0,1.0
1026,0.400630,-0.856888,0.676200,0.702626,-1.996964,0.496893,-0.276474,0.0,1.0
1027,-1.270229,0.759579,0.521589,-0.017528,-1.037957,0.080107,-0.276474,1.0,0.0
1028,-1.168265,1.308065,-0.279579,0.853356,0.213257,0.191166,-0.276474,0.0,1.0


In [69]:
# Train Test Split 
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X_pre, Y, test_size=20, random_state=23)

In [70]:
xtrain

,num__Cement,num__Blast Furnace Slag,num__Water,num__Superplasticizer,num__Coarse Aggregate,num__Fine Aggregate,num__Age,cat__Fly Ash_No,cat__Fly Ash_Yes
925,-1.121131,1.033242,0.723052,0.300680,-0.154899,-1.654422,-0.276474,0.0,1.0
937,0.315019,-0.856888,-0.143708,0.300680,0.371590,-1.059192,-0.276474,0.0,1.0
841,0.235180,-0.856888,0.676200,0.635635,-0.180644,-0.855790,-0.276474,1.0,0.0
606,-0.428547,-0.856888,0.582497,-1.039143,-0.064791,1.390362,-0.501591,1.0,0.0
345,-0.643055,-0.856888,-1.254098,0.669131,1.035814,0.035184,-0.501591,0.0,1.0
...,...,...,...,...,...,...,...,...,...
237,-0.642093,0.280669,0.006218,0.082958,1.196721,0.148739,0.173758,1.0,0.0
31,-0.139970,0.465044,2.175461,-1.039143,-0.528203,-1.292542,5.142396,0.0,1.0
950,0.441993,-0.856888,0.816756,0.769617,-2.213223,0.236089,-0.276474,1.0,0.0
742,1.918544,-0.856888,0.488793,-1.039143,-0.476713,-0.656132,0.720469,0.0,1.0


In [71]:
# Create the Model 
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(xtrain, ytrain)

LinearRegression()

In [72]:
model.score(xtrain, ytrain)

0.5886124662274534

In [73]:
model.score(xtest, ytest)

0.7047169347955418

In [74]:
# Predict Train and Test Data 
yts_pre = model.predict(xtest)

In [76]:
yts_pre[0:5]

array([[37.28661314],
       [17.74093968],
       [57.0276142 ],
       [27.08246536],
       [19.62361798]])

In [77]:
ytest.head()

,Strength
874,36.80
184,11.58
158,71.70
264,19.52
575,17.58


In [79]:
ytr_pre = model.predict(xtrain)

In [80]:
ytr_pre[0:5]

array([[31.10637764],
       [37.16572277],
       [32.83869916],
       [16.01916968],
       [32.25377905]])

In [81]:
ytest.head()

,Strength
874,36.80
184,11.58
158,71.70
264,19.52
575,17.58
